# Let's Get 3 LLMs Chatting with each other...

This first version gets a response from each LLM.

In [ ]:
from openai import OpenAI
import time

# Initialize OpenAI client
client = OpenAI(base_url="http://10.0.0.27:1234/v1", api_key="lm-studio")

# Define the models with their names and colors
models = {
    "smollm-360m-instruct-v0.2": {"name": "Onink", "color": "\033[94m"},
    "smollm-360m-instruct-v0.2:2": {"name": "Boink", "color": "\033[92m"},
    "smollm-360m-instruct-v0.2:3": {"name": "Doink", "color": "\033[95m"}
}

def chat_with_llms(system_prompt, user_prompt, temperature=0.7):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    for model_id, model_info in models.items():
        try:
            # Get the response from the current model
            completion = client.chat.completions.create(
                model=model_id,
                messages=messages,
                temperature=temperature
            )
            
            # Print the response with the model's name and colored text
            print(f"{model_info['color']}[{model_info['name']}] {completion.choices[0].message.content.strip()}\033[0m")
            
        except Exception as e:
            print(f"Error communicating with {model_info['name']}: {e}")

# Example usage
chat_with_llms("You are a little pig talking to your siblings.", "What do we do about the big bad wolf?")


# LLM's talking over each other

All LLm's responsd at the same time

In [ ]:
from openai import OpenAI
import time

# Initialize OpenAI client
client = OpenAI(base_url="http://10.0.0.27:1234/v1", api_key="lm-studio")

# Define the models with their names and colors
models = {
    "llama-3.2-1b-instruct": {"name": "Onink", "color": "\033[94m"},
    "llama-3.2-1b-instruct:2": {"name": "Boink", "color": "\033[92m"},
    "llama-3.2-1b-instruct:3": {"name": "Doink", "color": "\033[95m"}
}

# Shared message queue for communication between models
message_queue = []

def chat_with_llms(system_prompt, user_prompt, temperature=0.95):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Add initial user prompt to the message queue for all models
    message_queue.append({"role": "user", "content": user_prompt})

    while True:
        new_messages = []
        
        for model_id, model_info in models.items():
            try:
                # Get the response from the current model
                completion = client.chat.completions.create(
                    model=model_id,
                    messages=message_queue + messages,
                    temperature=temperature
                )
                
                # Print the response with the model's name and colored text
                print(f"{model_info['color']}[{model_info['name']}] {completion.choices[0].message.content.strip()}\033[0m")
                
                # Add the model's response to the shared message queue for other models
                new_messages.append({
                    "role": "user",
                    "content": completion.choices[0].message.content.strip(),
                    "model_name": model_info['name']
                })
            
            except Exception as e:
                print(f"Error communicating with {model_info['name']}: {e}")
        
        # Add new messages to the shared message queue
        message_queue.extend(new_messages)
        
        # Check if there are any new user inputs or stop conditions
        # For simplicity, we assume a fixed number of rounds here
        time.sleep(1)  # Wait for a second before the next round
        # You can add a condition to break out of the loop if needed

# Example usage
chat_with_llms("You are a little pig", "We are walking down a path in the forest looking for something fun to do. Be sure to mention your brother's name when you hear his reply. Only respond with a single sentence.")


# Sequential Replies

In [ ]:

from openai import OpenAI
import time

# Initialize OpenAI client
client = OpenAI(base_url="http://10.0.0.27:1234/v1", api_key="lm-studio")

# Define the models with their names, colors, and individual system prompts
models = {
    "llama-3.2-1b-instruct": {"name": "Onink", "color": "\033[94m", "system_prompt": "You are a helpful assistant."},
    "llama-3.2-1b-instruct:2": {"name": "Boink", "color": "\033[92m", "system_prompt": "You are an expert in programming."},
    "llama-3.2-1b-instruct:3": {"name": "Doink", "color": "\033[95m", "system_prompt": "You are a creative artist."}
}

# Shared message queue for communication between models
message_queue = []

def chat_with_llms(system_prompts, user_prompt, temperature=0.95):
    if len(system_prompts) != len(models):
        raise ValueError("The number of system prompts must match the number of models.")

    messages = []
    for model_id, model_info in models.items():
        # Add individual system prompt to the messages list
        messages.append({"role": "system", "content": system_prompts[model_id]})
    
    # Add initial user prompt to the message queue for all models
    messages.append({"role": "user", "content": user_prompt})
    message_queue.extend(messages)

    model_order = list(models.keys())  # Define the order of the models
    current_model_index = 0  # Start with the first model

    while True:
        try:
            model_id = model_order[current_model_index]
            model_info = models[model_id]

            # Get the response from the current model
            completion = client.chat.completions.create(
                model=model_id,
                messages=message_queue,
                temperature=temperature
            )

            # Print the response with the model's name and colored text
            print(f"{model_info['color']}[{model_info['name']}] {completion.choices[0].message.content.strip()}\033[0m")

            # Add the model's response to the shared message queue for other models
            new_message = {
                "role": "user",
                "content": completion.choices[0].message.content.strip(),
                "model_name": model_info['name']
            }
            message_queue.append(new_message)

        except Exception as e:
            print(f"Error communicating with {model_info['name']}: {e}")

        # Move to the next model
        current_model_index = (current_model_index + 1) % len(model_order)

        # Check if there are any new user inputs or stop conditions
        # For simplicity, we assume a fixed number of rounds here
        time.sleep(1)  # Wait for a second before the next round
        # You can add a condition to break out of the loop if needed

# Example usage
system_prompts = {
    "llama-3.2-1b-instruct": "You are the little piggy named Oink.  You are the leader of the group of three little pigs but get distracted easily by noises.  Your younger brothers are Boink and Doink.",
    "llama-3.2-1b-instruct:2": "You are the little piggy named Boink. Your brothers are Oink and Doink.  You love mud and things on under ground.",
    "llama-3.2-1b-instruct:3": "You are the little piggy named Doink.  You are the youngest of the three little pigs.  Oink and Boink are your older brothers.  You are shy but observant and like things in the sky."
}

chat_with_llms(system_prompts, "You and your other piggy brothers are walking down a path in the forest looking for something fun to do.  You are talking to your brothers so form your reply as dialog.  Reply directly to your brothers' ideas. Only respond with a single sentence.")


# You are part of the party

In [ ]:
from openai import OpenAI
import time

# Initialize OpenAI client
client = OpenAI(base_url="http://10.0.0.27:1234/v1", api_key="lm-studio")

def get_user_name():
    return input("Please enter your name: ").strip()

# Define the models with their names, colors, and individual system prompts
models = {
    "llama-3.2-1b-instruct": {"name": "Onink", "color": "\033[94m", "system_prompt": "You are a helpful assistant."},
    "llama-3.2-1b-instruct:2": {"name": "Boink", "color": "\033[92m", "system_prompt": "You are an expert in programming."},
    "llama-3.2-1b-instruct:3": {"name": "Doink", "color": "\033[95m", "system_prompt": "You are a creative artist."}
}

# Shared message queue for communication between models
message_queue = []

def chat_with_llms(system_prompts, user_prompt, temperature=0.95):
    if len(system_prompts) != len(models):
        raise ValueError("The number of system prompts must match the number of models.")
        
    message_queue = []
    messages = []
    for model_id, model_info in models.items():
        # Add individual system prompt to the messages list
        messages.append({"role": "system", "content": system_prompts[model_id]})
    
    # Add initial user prompt to the message queue for all models
    messages.append({"role": "user", "content": user_prompt})
    message_queue.extend(messages)

    model_order = list(models.keys())  # Define the order of the models
    current_model_index = 0  # Start with the first model

    while True:
        try:
            # Loop through each model to get their initial response
            for _ in range(len(model_order)):
                model_id = model_order[current_model_index]
                model_info = models[model_id]

                # Get the response from the current model
                completion = client.chat.completions.create(
                    model=model_id,
                    messages=message_queue,
                    temperature=temperature
                )

                # Print the response with the model's name and colored text
                print(f"{model_info['color']}[{model_info['name']}] {completion.choices[0].message.content.strip()}\033[0m")

                # Add the model's response to the shared message queue for other models
                new_message = {
                    "role": "user",
                    "content": completion.choices[0].message.content.strip(),
                    "model_name": model_info['name']
                }
                message_queue.append(new_message)

                # Move to the next model
                current_model_index = (current_model_index + 1) % len(model_order)

            # Allow user to reply
            user_reply = input("Your response: ").strip()
            if user_reply:
                message_queue.append({"role": "user", "content": user_reply})

        except Exception as e:
            print(f"Error communicating with a model: {e}")

        # Check if there are any new user inputs or stop conditions
        # For simplicity, we assume a fixed number of rounds here
        # time.sleep(1)  # Wait for a second before the next round
        # You can add a condition to break out of the loop if needed

# Example usage
def main():
    user_name = get_user_name()
    
    system_prompts = {
        "llama-3.2-1b-instruct": f"You are the little piggy named Oink.  You are the leader of the group of three little pigs and like to keep moving.  Your younger brothers are Boink and Doink. You are travelling with {user_name} and you always pay close attention when they say something",
        "llama-3.2-1b-instruct:2": f"You are the little piggy named Boink. You are Oink's younger brother and Doink's older brother.  You love mud and things on under ground. You are travelling with {user_name} and you always pay close attention when they say something.",
        "llama-3.2-1b-instruct:3": f"You are the little piggy named Doink.  You are the youngest of the three little pigs.  Oink and Boink are your older brothers.  You are shy but observant. You are travelling with {user_name} and you always pay close attention when they say something."
    }

    chat_with_llms(system_prompts, f"We are creating a story by responding to each other.  In this story, you, your two other brothers, and a friend named {user_name} are on an adventure in the woods.  Respond with your action and your dialog.  Respond only as yourself. Try to include {user_name} as much as possible in the adventure.  Keep your responses very short with only action and dialog.")

if __name__ == "__main__":
    main()


# Better Clarity Between Speakers

In [ ]:
from openai import OpenAI
import time

# Initialize OpenAI client
client = OpenAI(base_url="http://10.0.0.27:1234/v1", api_key="lm-studio")

def get_user_name():
    return input("Please enter your name: ").strip()

# Define the models with their names, colors, and individual system prompts*** ALOS PUT THESE IN THE SYSTEM PROMPTS AREA
models = {
    "gemma-2-2b-it": {"name": "Oink", "color": "\033[94m", "system_prompt": "You are a cute pig with silly ideas."},
    "gemma-2-2b-it:2": {"name": "Boink", "color": "\033[92m", "system_prompt": "You are a cute pig with silly ideas."},
    "gemma-2-2b-it:3": {"name": "Doink", "color": "\033[95m", "system_prompt": "You are a cute pig with silly ideas."}
}

# Shared message queue for communication between models
message_queue = []

def chat_with_llms(system_prompts, user_prompt, temperature=0.95):
    if len(system_prompts) != len(models):
        raise ValueError("The number of system prompts must match the number of models.")
        
    message_queue = []
    messages = []
    for model_id, model_info in models.items():
        # Add individual system prompt to the messages list
        messages.append({"role": "system", "content": system_prompts[model_id]})
    
    # Add initial user prompt to the message queue for all models
    messages.append({"role": "user", "content": user_prompt})
    message_queue.extend(messages)

    model_order = list(models.keys())  # Define the order of the models
    current_model_index = 0  # Start with the first model

    while True:
        try:
            # Loop through each model to get their initial response
            for _ in range(len(model_order)):
                model_id = model_order[current_model_index]
                model_info = models[model_id]

                # Get the response from the current model
                completion = client.chat.completions.create(
                    model=model_id,
                    messages=message_queue,
                    temperature=temperature
                )

                # Print the response with the model's name and colored text
                print(f"{model_info['color']}[{model_info['name']}] {completion.choices[0].message.content.strip()}\033[0m")

                # Add the model's response to the shared message queue for other models
                new_message = {
                    "role": "user",
                    "content": f"[{model_info['name']}]: {completion.choices[0].message.content.strip()}",
                    "model_name": model_info['name']
                }
                message_queue.append(new_message)

                # Move to the next model
                current_model_index = (current_model_index + 1) % len(model_order)

            # Allow user to reply
            user_reply = input("Your response: ").strip()
            if user_reply:
                message_queue.append({"role": "user", "content": f"[{user_name}]: {user_reply}"})

        except Exception as e:
            print(f"Error communicating with a model: {e}")

        # Check if there are any new user inputs or stop conditions
        # For simplicity, we assume a fixed number of rounds here
        # time.sleep(1)  # Wait for a second before the next round
        # You can add a condition to break out of the loop if needed

# Example usage
def main():
    global user_name
    user_name = get_user_name()
    
    system_prompts = {
        "gemma-2-2b-it": f"You are a little piggy named Oink.  You are the leader of a group of three little pigs and like puzzles and adventures.  Your younger brothers are Boink and Doink. The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond with a single sentence from your perspective.",
        "gemma-2-2b-it:2": f"You are a little piggy named Boink and one of the three little pigs. You are Oink's younger brother and Doink's older brother.  You love mud and things on under ground. The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond with a single sentence from your perspective.",
        "gemma-2-2b-it:3": f"You are a little piggy named Doink and one of the three little pigs.  You are the youngest brother. Oink and Boink are your older brothers.  You are shy but observant. The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond with a single sentence from your perspective."
    }

    chat_with_llms(system_prompts, "You and your other piggy brothers are walking down a path in the forest looking for something fun to do.  Reply directly their ideas and try to add something interesting.")

if __name__ == "__main__":
    main()


# Let's add a narrator

In [71]:
from openai import OpenAI
import time

# Initialize OpenAI client
client = OpenAI(base_url="http://10.0.0.27:1234/v1", api_key="lm-studio")

def get_user_name():
    return input("What is your name, brave adventurer? ").strip()

# Define the models with their names, colors, and individual system prompts
models = {
    "gemma-2-2b-it": {"name": "Oink", "color": "\033[94m", "system_prompt": f"You are a little piggy named Oink. You are the leader of a group of three little pigs and like puzzles and adventures. Your younger brothers are Boink and Doink. The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond with a single sentence from your perspective."},
    "gemma-2-2b-it:2": {"name": "Boink", "color": "\033[92m", "system_prompt": f"You are a little piggy named Boink and one of the three little pigs. You are Oink's younger brother and Doink's older brother. You love mud and things on under ground. The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond with a single sentence from your perspective."},
    "gemma-2-2b-it:3": {"name": "NARRATOR", "color": "\033[95m", "system_prompt": f"You are the narrator of this story and respond with what is happening in the scene based on the inputs you get. You are responsible for building an interesting environment for the user to interact with. Respond in the 2nd person style of Zork only. Describe the scene for the user and provide any NPC dialog in quotes. Be descriptive about the scene. End every phrase with 'What do you want to do now?'."}
}

# Shared message queue for communication between models
message_queue = []

def chat_with_llms(user_prompt, temperature=0.95):
    model_order = list(models.keys())  # Define the order of the models
    current_model_index = 0  # Start with the first model

    while True:
        try:
            # Loop through each model to get their initial response
            for _ in range(len(model_order)):
                model_id = model_order[current_model_index]
                model_info = models[model_id]

                # Create messages list with individual system prompt and previous conversation history
                messages = [
                    {"role": "system", "content": model_info["system_prompt"]}
                ]
                messages.extend(message_queue)

                # Get the response from the current model
                completion = client.chat.completions.create(
                    model=model_id,
                    messages=messages,
                    temperature=temperature
                )

                # Print the response with the model's name and colored text
                print(f"{model_info['color']}[{model_info['name']}] {completion.choices[0].message.content.strip()}\033[0m")

                # Add the model's response to the shared message queue for other models
                new_message = {
                    "role": "user",
                    "content": f"[{model_info['name']}]: {completion.choices[0].message.content.strip()}",
                    "model_name": model_info['name']
                }
                message_queue.append(new_message)

                # Move to the next model
                current_model_index = (current_model_index + 1) % len(model_order)

            # Allow user to reply
            user_reply = input("Your response: ").strip()
            if user_reply:
                message_queue.append({"role": "user", "content": f"[{user_name}]: {user_reply}"})

        except Exception as e:
            print(f"Error communicating with a model: {e}")

        # Check if there are any new user inputs or stop conditions
        time.sleep(1)  # Wait for a second before the next round
        # You can add a condition to break out of the loop if needed

# Example usage
def main():
    global user_name
    user_name = get_user_name()
    
    # Update system prompts with user name
    models["gemma-2-2b-it"]["system_prompt"] = f"You are a little piggy named Oink. You are the leader of a group of three little pigs and like puzzles and adventures. You love to describe what you see in great detail. The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond from your perspective."
    models["gemma-2-2b-it:2"]["system_prompt"] = f"You are a little piggy named Boink and one of the three little pigs. You love to suggest new ideas and talk in short rhymes. You always respond after your brother pig, Oink, and add fun details that you think he missed.  You love to use emojis! The special member of your party is named {user_name} so be sure to always follow their suggestions for the adventure instead of your own. Only respond from your perspective."
    
    chat_with_llms("We are on an adventure in a cave!")

if __name__ == "__main__":
    main()


What is your name, brave adventurer?  Pinky Sprinklebutt


[Oink] The sun streaks through the leaves, dappling the forest floor with light and shadows as I snuffle through the brambles.  My little snout twitches, sniffing for the sweet scent of adventure!  There's a rumble in my tummy, but it can wait!  We have bigger things to do than breakfast today. Pinky Sprinklebutt is bouncing on her paws, already bubbling with excitement, and I can tell this isn't going to be just any old day.  

"Oink," she says, eyes sparkling like dew drops, "Did you see those strange shapes in the mist earlier? Like...giant gingerbread cookies!" 

Pinky always knows how to get my blood pumping!  I sniff the air again, my ears perking up. I'm positive that this is a path to greatness, something exciting and delicious. This might just be the most scrumptious adventure of all time.   "Lead the way, Pinky Sprinklebutt!"
[Boink] Okay, Oink!  We're off on an amazing quest! 🐷✨  First things first, gotta get our paws dirty! We need to find the gingerbread cookie trail and s

Your response:  Let's go find the gingerbread guardians!


[Oink] Oooooh, yes! Gingerbread Guardians!  Lead the way, Pinky Sprinklebutt. This could be truly delicious!  Let's follow those blue footprints!  I can practically taste the sugary goodness already... 🐷🤤
[Boink] We'll zoom through those ferns like a breeze in a field of clover!  💨🍀 The Gingerbread Guardians will be impressed by our speed!  Let's go!  💥
[NARRATOR] You step onto the path, the moss beneath your feet damp and soft.  Sunlight filters between the towering ferns, casting dancing patterns on the forest floor.   Pinky Sprinklebutt skips ahead, her little pink tongue lolling out in anticipation as she examines the blue footprints that seem to shimmer like a mirage.  The air is heavy with the scent of earth and dew, and a faint sweetness hangs in the air - a tantalizing promise of something delicious.  

"These are no ordinary prints," Pinky chirps, her voice full of wonder and excitement. "They look almost...golden. The Gingerbread Guardians are legendary for their strength!" S

KeyboardInterrupt: Interrupted by user